In [ ]:
import sys
sys.path.insert(0, '/Users/nina/Documents/treemap/treemap/analysis')
from config_figures import *

### Range size dataframe

In [ ]:
df = pd.read_csv('../../data/sdms_area_lat_elev.csv').drop(['system:index', '.geo'], axis=1)
df

### Biome dataframe

In [ ]:
biome_names = {
    'Boreal Forests/Taiga': 'Boreal Forests',
    'Mediterranean Forests, Woodlands & Scrub': 'Mediterranean Forests',
    'Temperate Broadleaf & Mixed Forests': 'Temperate Broadleaf Forests', 
    'Temperate Conifer Forests': 'Temperate Conifer Forests',
    'Tropical & Subtropical Coniferous Forests': 'Tropical Coniferous Forests',
    'Tropical & Subtropical Dry Broadleaf Forests': 'Tropical Dry Broadleaf Forests',
    'Tropical & Subtropical Moist Broadleaf Forests': 'Tropical Moist Broadleaf Forests'
}

biome_df = pd.read_csv('../../data/' + sdm_biome_drive_filename + '.csv').drop(['system:index', '.geo'], axis=1)
biome_df = pd.melt(biome_df, id_vars=['species'], var_name = 'biome', value_name = 'sdm_frac')
biome_df = biome_df[(biome_df['sdm_frac'] > 20) & (biome_df['biome'].str.contains('Forests'))]
biome_df['biome_name'] = biome_df['biome'].apply(lambda b: biome_names[b])
biome_df

### Construct dataframe with range size and range restriction per species with biome attributed to each species

In [ ]:
df_pivot = df[df['climate'] == '1981_2010'].pivot(index='species', columns='min_tree_cover', values='area').reset_index()
df_pivot[10] = 100 - (df_pivot[10] / df_pivot[0] * 100)
df_pivot[20] = 100 - (df_pivot[20] / df_pivot[0] * 100)
df_pivot[0] = np.nan
df_range_restriction = df_pivot.melt(id_vars='species', value_vars=[0,10,20], value_name='range_restriction')

df_range_size_restriction = df[df['climate'] == '1981_2010'][['species', 'min_tree_cover', 'area']].merge(df_range_restriction).merge(biome_df[['species', 'biome_name']], how='outer')
df_range_size_restriction['area'] = df_range_size_restriction['area'] / 1e12
df_range_size_restriction['area_log10'] = np.log10(df_range_size_restriction['area'])
df_range_size_restriction

In [ ]:
sns.set(style='white', font_scale = 1.2)
fig, axes = plt.subplots(7, 2, figsize=(16, 6))
biome_order = [
    'Tropical Coniferous Forests', 'Tropical Moist Broadleaf Forests', 'Tropical Dry Broadleaf Forests', 
    'Boreal Forests', 'Temperate Conifer Forests', 'Temperate Broadleaf Forests', 'Mediterranean Forests'
]
palette=['#F0E442', '#56B4E9', '#E69F00', '#009E73', '#CC79A7', '#0072B2', '#D55E00']

for i, ((ax1, ax2), biome, color) in enumerate(zip(axes, biome_order, palette)):
    print(i, biome, ax1, ax2)
    df_biome_0 = df_range_size_restriction[(df_range_size_restriction['biome_name'] == biome) & (df_range_size_restriction['min_tree_cover'] == 0)]
    df_biome_10 = df_range_size_restriction[(df_range_size_restriction['biome_name'] == biome) & (df_range_size_restriction['min_tree_cover'] == 10)]
    sns.kdeplot(ax=ax1, data=df_biome_0[df_biome_0['area'] != 0], x='area', color='black', fill=True, alpha = 0.3, log_scale=True)
    sns.kdeplot(ax=ax1, data=df_biome_10[df_biome_10['area'] != 0], x='area', color=color, fill=True, alpha = 0.3, log_scale=True)
    ax1.set(xlim=(0.0005, 15))
    ax1.set_ylabel(biome, rotation=0)

    sns.barplot(ax=ax2, data=df_biome_10, x='range_restriction', y='biome_name', color=color, errorbar=("ci", 95), estimator='mean')
    ax2.set(ylabel="", yticklabels="", xlim=(0,100))

    if i != 6: 
        for ax in [ax1, ax2]:
            ax.set(xticks=[], yticks=[], xlabel="")
            for side in ['top', 'bottom', 'right', 'left']:
                if ax==ax1 and side=='bottom': continue
                ax.spines[side].set_visible(False)
    else:
        ax1.set(xlabel="SDM range size (million km2)", yticklabels="")
        ax2.set(xlabel="SDM range reduction (%)")
        for ax in [ax1, ax2]:
            for side in ['top', 'right', 'left']:
                ax.spines[side].set_visible(False)

plt.savefig('../../figures/range_restriction_figure.png')

### Statistics on range size restriction per biome 

In [ ]:
df_range_size_restriction[df_range_size_restriction['min_tree_cover'] == 10].range_restriction.mean()

In [ ]:
stats = df_range_size_restriction[df_range_size_restriction['min_tree_cover'].isin([0, 10])].groupby(
    ['biome_name', 'min_tree_cover'])[['area','range_restriction']].mean()
stats

In [ ]:
stats = df_range_size_restriction[df_range_size_restriction['min_tree_cover'] == 10].groupby(
    ['biome_name', 'min_tree_cover'])['range_restriction'].agg(['mean','sem'])
stats['C95_low'] = stats['mean'] - 1.96 * stats['sem'] 
stats['C95_high'] = stats['mean'] + 1.96 * stats['sem']
stats

### SDM range size vs. median latitude

In [ ]:
df_area_lat = df[(df['min_tree_cover'] == 10) & (df['climate'] == '1981_2010') & (df['median_lat'] != -999)][['species', 'area', 'median_lat']]
df_area_lat['area'] = df_area_lat['area'] / 1e12
df_area_lat

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
values = np.vstack([df_area_lat["area"], df_area_lat["median_lat"]])
kernel = gaussian_kde(values)(values)
kernel_norm = (kernel - kernel.min()) / (kernel.max() - kernel.min())

In [ ]:
latitudes = np.arange(-55, 65, 1)
mean_area = [df_area_lat[(df['median_lat'] >= lat-2) & (df['median_lat'] < lat+2)]['area'].mean() for lat in latitudes]

In [ ]:
sns.set_style("white")
fig, ax = plt.subplots(figsize=(4,6))
sns.scatterplot(ax=ax, data = df_area_lat, x = 'area', y = 'median_lat', hue = kernel_norm, palette = 'turbo', s = 5, edgecolor = 'none', legend=False)
sns.lineplot(ax=ax, x = mean_area, y = latitudes, orient='y', c='k')
ax.set(ylim=(-40, 70))
ax.set_xlabel('SDM range size (million km2)', fontsize=14)
ax.set_ylabel('SDM median latitude', fontsize = 14)
plt.tight_layout()
plt.savefig('../figures/sdm_range_median_lat.png')